In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import StratifiedKFold

## Loading datasets

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")
sample_sub = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")

In [ ]:
# looking at the shape of dataset
train.shape , test.shape , sample_sub.shape

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
# Finding null values / Missing values
train.isnull().sum().sum()

## Corelation matrix

In [ ]:
# Compute the correlation matrix
corr = train.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
cols = ['f'+str(i) for i in range(100)]

## Trellis plots

In [ ]:
# plot the first 32 features 
i = 1
plt.figure()
fig, ax = plt.subplots(8, 4,figsize=(20, 22))
for feature in cols[:32]:
    plt.subplot(8, 4,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
sns.catplot(x="target", kind="count", palette="ch:.25", data=train)

## Target Feature Distribution

In [ ]:
## Target distibution
pie, ax = plt.subplots(figsize=[18,8])
train.groupby('target').size().plot(kind='pie',autopct='%.1f',ax=ax,title='Target distibution')

### Transformation of Data

In [ ]:
# Train test split

from sklearn.model_selection import train_test_split

random_state_val =42
test_size_val =0.2
train,validation = train_test_split(train, test_size  = test_size_val, random_state = random_state_val)

In [ ]:
y_nm = 'target'

df_train_x = train.drop(y_nm, axis = 1)
df_train_y = pd.DataFrame(train[y_nm])

df_val_x = validation.drop(y_nm, axis = 1)
df_val_y = pd.DataFrame(validation[y_nm])


df_test_x = test

In [ ]:
num_cols = [col for col in df_train_x.columns if df_train_x[col].dtype in ["float16","float32","float64"]]
cat_cols = [col for col in df_train_x.columns if df_train_x[col].dtype not in ["float16","float32","float64"]]

In [ ]:
y = train["target"].copy()

for cols in cat_cols:
    enc = TargetEncoder(cols=[cols])
    df_train_x = enc.fit_transform(df_train_x, y)
    df_val_x = enc.transform(df_val_x)
    df_test_x = enc.transform(df_test_x)

## Normalize data

In [ ]:
scaler = QuantileTransformer()
scaler.fit(df_train_x)
df_train_x = pd.DataFrame(scaler.transform(df_train_x))
df_val_x = pd.DataFrame(scaler.transform(df_val_x))
df_test_x = pd.DataFrame(scaler.transform(df_test_x))

## Model XGB Classifier

In [ ]:
XGBClassifier = xgb.XGBClassifier(max_depth = 8,
                                 learning_rate = 0.005,
                                 n_estimators = 10000,
                                 objective = 'binary:logistic',
                                 tree_method = 'gpu_hist',
                                 booster = 'gbtree',
                                 gamma = 0.64,
                                 max_delta_step = 3,
                                 min_child_weight = 7,
                                 subsample = 0.7,
                                 colsample_bytree = 0.8,
                                 n_jobs = -1
                                 )

In [ ]:
start = datetime.datetime.now()
xgb = XGBClassifier.fit(df_train_x.values,
                       df_train_y.values.ravel(),
                       eval_set = [(df_train_x.values, df_train_y), (df_val_x.values, df_val_y)], 
                       eval_metric = 'auc',
                       early_stopping_rounds = 15,
                       verbose = True)
end = datetime.datetime.now()
end-start

In [ ]:
fi_vals = xgb.get_booster().get_score(importance_type = 'weight')
fi_dict = {df_train_x.columns[i]:float(fi_vals.get('f'+str(i),0.)) for i in range(len(df_train_x.columns))}
feature_importance_ = sorted(fi_dict.items(), key=operator.itemgetter(1), reverse=True)
feature_importance_result = OrderedDict(feature_importance_)

importance = pd.DataFrame(feature_importance_)
importance.columns = ['feature','weight']
importance.head(10)

## 10 Most important features

In [ ]:
importance_ten = importance[:10]
importance_ten.set_index('feature').sort_values(by='weight').plot(kind='barh', figsize=(5, 5))

In [ ]:
fpr, tpr, _ = roc_curve(df_val_y, xgb.predict_proba(df_val_x.values)[:, 1])
roc_auc = auc(fpr, tpr)

result_lst =[]
max_roc_auc =0.
opt_threshold =0.
val_y_prob = xgb.predict_proba(df_val_x.values)[:, 1]

for n in range(0,50):
    threshold = round(((n+1)*0.01),2)
    pred_yn = val_y_prob.copy()
    pred_yn = np.where(pred_yn > threshold, 1., 0.)
    
    result_dict = {}
    precision, recall, f1_score, support = precision_recall_fscore_support(df_val_y.values.ravel(), pred_yn, average='binary')
    accuracy = accuracy_score(df_val_y.values.ravel(), pred_yn)
    kappa = cohen_kappa_score(df_val_y.values.ravel(), pred_yn)
    
    result_dict ={'Threshold': threshold, 'Accuracy': round(accuracy,4), 'Precision': round(precision,4), 'Recall': round(recall,4), 'F1_Score': round(f1_score,4),'roc_auc': round(roc_auc,4), 'Kappa': round(kappa,4)}
    result_lst.append(result_dict)
    
    if max_roc_auc <= roc_auc:
        max_roc_auc = roc_auc
        opt_threshold = threshold
        
    confMat = confusion_matrix(df_val_y.values.ravel(), pred_yn, labels=[1,0])
    
matric_df = pd.DataFrame(result_lst, columns=['Threshold','Accuracy', 'Precision', 'Recall', 'F1_Score','roc_auc' ,'Kappa'])
matric_df.to_csv('REC_scores.csv',sep=',', header=True, index=False, encoding='UTF-8')

print('Max roc_auc =%f, optimized_threshold=%f'%(max_roc_auc, opt_threshold))
print('Complete')

In [ ]:
predict_xgb = xgb.predict_proba(df_train_x.values)[:,1]
pred_train = np.where(predict_xgb > opt_threshold, 1., 0.)

tp, fn, fp, tn = confusion_matrix(df_train_y.values.ravel(), pred_train, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(df_train_y.values.ravel(), pred_train),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(df_train_y.values.ravel(), pred_train))

## ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_train_y.values.ravel(), predict_xgb)

import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
pred_test = xgb.predict_proba(df_test_x.values)[:,1]

test_result= pd.DataFrame(pred_test)
test_result.columns = ['target']
predict = test_result['target']
Id_No = test['id']
submission = pd.DataFrame({'id': Id_No, 'target': predict})
submission['target'] = submission['target'].astype('float32')
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
print("Up Vote if you like the Code")